In [7]:
using(BenchmarkTools)

In [11]:
## Stillou Webou
const coeff = 4.1
function three_body_energy(x0, x1, x2, x3, x4, x5)
    u0 = -x0 + x2
    u1 = -x1 + x3
    u2 = -x2 + x4
    u3 = -x3 + x5
    return coeff*(-1 + (u0*u2 + u1*u3)/sqrt( (u0^2 + u1^2)*(u2^2 + u3^2) ))^2
    end;

function three_body_gradient_old(x0, x1, x2, x3, x4, x5)
    u0 = -x0 + x2
    u1 = -x2
    u2 = u1 + x4
    u3 = -x1 + x3
    u4 = -x3
    u5 = u4 + x5
    u6 = u0*u2 + u3*u5
    u7 = u0^2 + u3^2
    u8 = 1/sqrt(u7)
    u9 = u2^2 + u5^2
    u10 = 1/sqrt(u9)
    u11 = u10*u8
    u12 = (u11*u6 - 1)*coeff
    u13 = x2 - x4
    u14 = 2*u11
    u15 = 2*u6
    u16 = u10*u15/u7^(3/2)
    u17 = x3 - x5
    u18 = u15*u8/u9^(3/2)
     
    return (u12*(u0*u16 + u13*u14), 
            u12*(u14*u17 + u16*u3), 
            u12*(u14*(x0 - 2*x2 + x4) + u16*(u1 + x0) + u18*u2), 
            u12*(u14*(x1 - 2*x3 + x5) + u16*(u4 + x1) + u18*u5), 
            u12*(u0*u14 + u13*u18), 
            u12*(u14*u3 + u17*u18) )
    end;

In [12]:
function three_body_gradient(x0, x1, x2, x3, x4, x5)
    u0 = -x0 + x2
    u1 = -x2
    u2 = u1 + x4
    u3 = -x1 + x3
    u4 = -x3
    u5 = u4 + x5
    u6 = u0*u2 + u3*u5
    u7 = u0^2 + u3^2
    u8 = 1/sqrt(u7)
    u9 = u2^2 + u5^2
    u10 = 1/sqrt(u9)
    u11 = u10*u8
    u12 = (u11*u6 - 1)*coeff
    u13 = x2 - x4
    u14 = 2*u11
    u15 = 2*u6
    u16 = u10*u15 *u8^3
    u17 = x3 - x5
    u18 = u15*u8 *u10^3
     
    return (u12*(u0*u16 + u13*u14), 
            u12*(u14*u17 + u16*u3), 
            u12*(u14*(x0 - 2*x2 + x4) + u16*(u1 + x0) + u18*u2), 
            u12*(u14*(x1 - 2*x3 + x5) + u16*(u4 + x1) + u18*u5), 
            u12*(u0*u14 + u13*u18), 
            u12*(u14*u3 + u17*u18) )
    end;

In [13]:
xi = rand(6)
three_body_energy(xi...)

0.6753653122599329

In [9]:
 @benchmark three_body_energy(xi...) setup=(xi = rand(6))

BenchmarkTools.Trial: 
  memory estimate:  112 bytes
  allocs estimate:  7
  --------------
  minimum time:     171.098 ns (0.00% GC)
  median time:      174.082 ns (0.00% GC)
  mean time:        196.623 ns (7.38% GC)
  maximum time:     82.834 μs (99.78% GC)
  --------------
  samples:          10000
  evals/sample:     746

In [21]:
 @benchmark three_body_gradient(xi...) setup=(xi = rand(6))

BenchmarkTools.Trial: 
  memory estimate:  160 bytes
  allocs estimate:  7
  --------------
  minimum time:     200.823 ns (0.00% GC)
  median time:      204.671 ns (0.00% GC)
  mean time:        233.091 ns (8.34% GC)
  maximum time:     99.158 μs (99.75% GC)
  --------------
  samples:          10000
  evals/sample:     606

In [20]:
xi = rand(6)
println(three_body_gradient_old(xi...) .- three_body_gradient(xi...))

(1.4710455076283324e-15, 0.0, 3.552713678800501e-15, 0.0, -3.552713678800501e-15, 0.0)


In [23]:
x1, y1, x2, y2, x3, y3 =  three_body_gradient_old(xi...)

(0.2273621063535715, 15.055168228663293, -21.390657930512145, 1.4496333749590469, 21.16329582415857, -16.50480160362234)

In [26]:
x1 + x3 + x2

-3.552713678800501e-15